In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import umap # dimensionality reduction
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import hdbscan
from deep_translator import GoogleTranslator

c:\anaconda\envs\machine_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Preparing Dataset

In [3]:
dataset=pd.read_csv("C:/Users/rbw19/Downloads/dataset.csv",on_bad_lines='skip')
dataset=dataset["title","contents","category","author_name"]

,Unnamed: 0,post_id,title,agency_id,post_url,category_id,image_url,contents,published_date_and_time,author_name,...,tweet_created_at,tweet_text,tweet_id,vdo_title,vdo_description,vdo_published_at,vdo_thumbnail,vdo_url,active_id,trending_count
0,0,10,سمو ولي العهد يغادر مصر بعد المشاركة في مؤتمري...,12.0,http://alwatan.kuwait.tt/articledetails.aspx?i...,1,http://watanmedia.seyasi.com/resources/media/i...,\n بحفظ الل...,2022-11-07T00:00:00,alwatan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,12,الأمانة العامة للمجلس الأعلى للتخطيط والتنمية ...,204.0,https://www.scpd.gov.kw/Default5_news.aspx,4,https://www.scpd.gov.kw/NewsOthers/original_so...,\r\n ...,2022-11-06T00:00:00,General Secretariat of Supreme Council for Pla...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,13,"\r\n ""التأم...",258.0,http://www.pifss.gov.kw/sites/Ar/Pages/NewsDet...,1,http://www.pifss.gov.kw/Lists/News/Attachments...,\n\n\n\n\n\n \n\n\n,2022-11-06T00:00:00,Public Institution for Social Security in Kuwait,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,14,تمديد طرح الممارسة رقم ص و م م 4/ 2022/2023م ...,271.0,https://www.nationalfund.gov.kw/ar/media-cente...,4,NaN,تمديد طرح الممارسة رقم ص و م م 4/ 2022/2023م ...,2022-11-06T00:00:00,The National Fund for SME Development,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,16,بسم اللّه الرحمن الرحيم,202.0,https://www.gpf.gov.kw/Ar/home.aspx,1,assets/img/home/logo_home.png,,2022-11-07T18:04:03,government performance follow up,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15547,15546,19622,الصحة: 80 % زيادة في مراجعي ال...,73.0,https://www.alaan.cc/article/381029/%D8%A7%D9%...,2,https://www.alaan.cc/media/images/74D6A83A-87E...,قال مدير منطقة الفروانية الصحية د. محمد الرشيد...,2022-11-08T17:15:00,alaan newspaper,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15548,15547,19630,وزير الداخلية يشرف على ضبط حوالي «10» ملايين ح...,11.0,https://sarmad.com/newsdetails.php?id=320039,3,https://sarmad.com/upload/news/320039_935.jpg,\n\t\t\t\t\t\t\t\t\t\n\nذكرت الإدارة العامة لل...,2022-11-08T18:41:00,sarmad news,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15549,15548,19639,الصندوق الكويتي للتنمية يشارك بجناح الكويت على...,11.0,https://sarmad.com/newsdetails.php?id=320034,1,https://sarmad.com/upload/news/320034_6de.jpg,\n\t\t\t\t\t\t\t\t\t\n\n(كونا) - شارك الصندوق ...,2022-11-08T17:14:00,sarmad news,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15550,15549,19641,«الصحة»: تخصصات طبية جديدة لخدمة فئة طريحي الف...,11.0,https://sarmad.com/newsdetails.php?id=320033,1,https://sarmad.com/upload/news/320033_2d6.jpg,وأوضحت الدكتورة الضبيب بقيام الفرق الطبية التا...,2022-11-08T17:09:00,sarmad news,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
